In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [4]:
from transformers import AutoTokenizer
import sys
import torch
import json
from scipy.special import softmax
from transformers import AutoModelForSequenceClassification
import argparse
import pandas as pd

In [5]:
# Load the test datasets from the GitHub repositories (access to them is obtained by request to the AGILE repository owner)

iptc_test = pd.read_json("../../datasets/IPTC-top-test/IPTC-top-test.jsonl", lines=True)
print(iptc_test.shape)

iptc_test.rename(columns={"label": "labels"}, inplace=True)

iptc_test.head(2)

(1129, 5)


,document_id,lang,text,labels,multilabel
0,CLASSLA-web.hr.3842863,hr,Iza kamere\n\nFilmsku družinu ZAG posjetila je...,"arts, culture, entertainment and media","[arts, culture, entertainment and media]"
1,CLASSLA-web.hr.4597011,hr,Učenici osmih razreda nedavno su u sklopu tere...,education,[education]


In [6]:
from transformers import pipeline

# Load a multi-class classification pipeline - if the model runs on CPU, comment out "device"
classifier = pipeline("text-classification", model="classla/multilingual-IPTC-news-topic-classifier", device=0, max_length=512, truncation=True)

In [7]:
def predict_topic(df_test_name):

    df = iptc_test

    texts = df["text"].to_list()

    prediction_list = []
    
    # Classify the texts
    results = classifier(texts)

    # Output the results
    for result in results:
        prediction_list.append(result["label"])
        
    # Create a json with results

    current_results = {
        "system": "IPTC XLM-R classifier",
        "predictions": [
            {
            "train": "EMMediaTopic (15k)",
            "test": "{}".format(df_test_name),
            "predictions": prediction_list,
            }
        ]
        }

    # Save the results as a new json
    with open("submissions/submission-{}-{}.json".format("IPTC-XLM-R-classifier", df_test_name), "w") as file:
        json.dump(current_results, file)

    print("Classification with {} on {} finished.".format("IPTC-XLM-R-classifier", df_test_name))

In [8]:
predict_topic("IPTC-test")

Classification with IPTC-XLM-R-classifier on IPTC-test finished.
